### 데이터 준비하기

In [ ]:
import os

dir_path = os.getenv('HOME')+'/aiffel/face_embedding/images'
file_list = os.listdir(dir_path)

print ("file_list: {}".format(file_list))

import matplotlib.pyplot as plt
import matplotlib.image as img

#Set figsize here
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(24,10))

# flatten axes for easy iterating
for i, ax in enumerate(axes.flatten()):
  image = img.imread(dir_path+'/'+file_list[i])
  ax.imshow(image)
plt.show()

fig.tight_layout()

In [ ]:
import face_recognition
import os
%matplotlib inline
import matplotlib.pyplot as plt

image_path = os.getenv('HOME')+'/aiffel/face_embedding/images/obama.jpg'
image = face_recognition.load_image_file(image_path)
face_locations = face_recognition.face_locations(image)

print(face_locations)  # 이미지에서 얼굴 영역의 좌표를 출력합니다.

a, b, c, d = face_locations[0]
cropped_face = image[a:c,d:b,:]

plt.imshow(cropped_face)   # 이미지에서 얼굴영역만 잘라낸 cropped_face를 그려 봅니다.

In [ ]:
import face_recognition
import os

def get_cropped_face(image_path):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
    
    return cropped_face

In [ ]:
image_path = os.getenv('HOME')+'/aiffel/face_embedding/image/

cropped_face = get_cropped_face(image_path)
plt.imshow(cropped_face)

## 얼굴 임베딩 만들기 (2)FaceNet

### 얼굴 임베딩 추출

In [ ]:
# import os

# dir_path = os.getenv('HOME')+'/aiffel/face_embedding/images'
# file_list = os.listdir(dir_path)

# print ("file_list: {}".format(file_list))

In [ ]:
image_file = os.path.join(dir_path, 'obama.jpg')
face = get_cropped_face(image_file)   # 얼굴 영역을 구하는 함수(이전 스텝에서 구현)

# 얼굴 영역을 가지고 얼굴 임베딩 벡터를 구하는 함수
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

embedding = get_face_embedding(face)  
embedding

In [ ]:
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        file_path = os.path.join(dir_path, file)
        face = get_cropped_face(file_path)
        embedding = get_face_embedding(face)
        
        if len(embedding) > 0:  
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
        
    return embedding_dict

In [ ]:
embedding_dict = get_face_embedding_dict(dir_path)
embedding_dict['trump']

## 5-5. 얼굴임베딩 사이의 거리측정

numpy.linalg.norm를 활용하여 L2 Norm Distance로 계산

In [ ]:
import numpy as np

def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

get_distance('obama', 'trump')

## 5-6. 얼굴임베딩 공간의 시각화

## 5-7. 가장 닮은꼴 얼굴 찾아보기

In [ ]:
def get_nearest_face(name, top=5): 
    pass
get_nearest_face('trump')

In [ ]:
# name1과 name2의 거리를 비교하는 함수를 생성하되, name1은 미리 지정하고, name2는 호출시에 인자로 받도록 합니다.
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

sort_key_func = get_sort_key_func('trump')   
# 이렇게 생성된 함수 sort_key_func는 sort_key_func('obama') 라고 호출할 때 trump와 obama 사이의 임베딩 벡터 거리를 계산합니다.

In [ ]:
sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))

sorted(정렬할 데이터, key 파라미터)
- sorted(data, key= 식) : 해당 키의 식을 기준으로 정렬하여 반환

In [ ]:
def get_nearest_face(name, top=5):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))
    
    for i in range(top+1):
        if i == 0 :   # 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))

In [ ]:
# obama와 가장 닮은 사람은 누굴까요?
get_nearest_face('obama')